# Overview

this notebook runs the models and save the time and score
the saved dataframe can be found in `../data/processed/results_{date}`

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
from src.graph import NetworkGraph

In [3]:
import time
import gurobi
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [4]:
from src.flows import *
from src.models.DQL.DQL import run

In [5]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator
from flatland.utils.rendertools import RenderTool

### Helper functions

In [6]:
def create_env(height,width,seed,number_of_agents=4):
    env = RailEnv(width=width,
              height=height,
              rail_generator=complex_rail_generator(nr_start_goal=20, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, seed = seed),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_of_agents)
    
    
    
    return env

In [7]:
def run_flow(env,height,width):
    
    #extract the transition matrix
    matrix_rail = np.array(env.rail.grid.tolist())
    
    #build the transition network
    flatlandNetwork = NetworkGraph(matrix_rail)

    #get the sources and sinks of the different agents
    sources = []
    sinks = []
    for agent in env.agents:
        sources.append(agent.initial_position)
        sinks.append(agent.target)
        
    #build the time exanded network and connect the sources and sinks
    TestNetworkTime = TimeNetwork(flatlandNetwork, depth=max(2*(height+width),40))
    TestNetworkTime.connect_sources_and_sink(sources,sinks)
    
    
    #get the Integral Program formulation of the problem and solve it 
    print(sources,sinks)
    mcflow = MCFlow(TestNetworkTime.graph,len(sources),TestNetworkTime.topology)
    mcflow.solve()
    
    #extract the path and compute the scores (total time spent by the agent in the grid)
    cost= 0
    if not mcflow.check_if_feasible():
        return "Infeasible"
    print(mcflow.m.objVal)
    paths = mcflow.extract_paths()
    for agent,path in paths.items():
        print(path)
        cost += -1+len(path)/2
       
    return cost

In [8]:
def run_flow_rl(env,width,height,seed,n_start_goal=20,initial_value = 0,
             learning_rate = 0.8,gamma = 0.9,epsilon = 0.1,threshold = 0.3):
    number_agents = len(env.agents)
    n_episodes = width*height*number_agents*15
    n_steps = 2*(width+height)
    print(seed)
    cost, n_episodes, total_time = run(number_agents,
                                    width,height,
                                    n_start_goal,
                                    seed,
                                    n_episodes,
                                    n_steps,
                                    initial_value,
                                    learning_rate,
                                    gamma,
                                    epsilon,
                                    threshold)
    
    return cost, n_episodes, total_time

In [9]:
def run_experiment_and_save_results(method, results,agents = 4, repetition = 5, 
                                    pathToSavedGrids = None, grids_size = None):
    nodes = []
    edges = []
    sizes_list = []
    if pathToSavedGrids is None:
        for sizes in tqdm(grids_size):
            for i in range(repetition):
                seed = sizes[0]*sizes[1]+i
                print(f'seed {seed}')
                env = create_env(sizes[0],sizes[1],seed,number_of_agents=agents)
                
                env.step({0:2})
                env_renderer = RenderTool(env,agent_render_variant=3)
                env_renderer.render_env(show=True, show_predictions=False, show_observations=False)
                env = create_env(sizes[0],sizes[1],seed,number_of_agents=agents)
    
                start = time.time()
                cost = run_flow(env,sizes[0],sizes[1])
                print(cost)
                sizes_list.append(sizes[0]*sizes[1])                    
                time_spent = time.time()-start
                results = results.append({'Size of the grid' : sizes[0]*sizes[1] , 
                                          'Score' : cost, "Time":time_spent, 
                                          "Type":"CO","Number of episodes": None} , 
                                         ignore_index=True)
                results.to_csv("../data/processed/results_comparison"+str(agents)+".csv")
                start = time.time()
                cost, n_episodes, total_time = run_flow_rl(env,sizes[1],sizes[0],seed)
                print(cost)
                sizes_list.append(sizes[0]*sizes[1])                    
                time_spent = time.time()-start
                results = results.append({'Size of the grid' : sizes[0]*sizes[1] , 
                                          'Score' : cost, "Time":time_spent, "Number of episodes": n_episodes,
                                          "Type":"RL"} , 
                                         ignore_index=True)
                results.to_csv("../data/processed/results_comparison.csv")
                print("---")
                
                env_renderer.close_window()
                
    else:
        raise NotImplementedError("Load the map and proceed to inference")

### Constant definition

In [10]:
grids_size = [(10,5),(10,10),(15,10),(15,15),(20,15),(20,20)]
grid_bigger = [(25,20),
              (25,25),(25,30),(30,30),(35,30), (35,35),(40,35),(40,40),
              (45,40),(45,45),(50,50),(55,50),(55,55),(60,55),(60,60),
              (65,60),(65,65),(70,65),(70,70),(75,70),(75,75),(80,75),
             (80,80),(85,90),(90,90),(95,90),(95,95),(100,95),(100,100)]

In [11]:
results = pd.DataFrame()

## Actual run

In [12]:
run_experiment_and_save_results(run_flow, results, agents = 1, repetition=10, grids_size=grids_size)

  0%|          | 0/6 [00:00<?, ?it/s]

seed 50
[(1, 2)] [(9, 0)]
Academic license - for non-commercial use only
Optimize a model with 29235 rows, 12997 columns and 48831 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 28689 rows and 11830 columns
Presolve time: 0.03s
Presolved: 546 rows, 1167 columns, 2332 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.165e+03
 Factor NZ  : 1.012e+04 (roughly 1 MByte of memory)
 Factor Ops : 2.164e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.05 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 288 iterations and 0.06 seconds
Optimal objective  1.900000000e+01
19.0
['source_agent_0', '(1, 2)_S_out_t0', '(2, 2)_N_in_t1', '(2, 2)_S_out_t2', '(3, 2)_N_in_t3'

KeyboardInterrupt: 

# PLOTS

In [ ]:
df = pd.read_csv("../data/processed/results_comparison.csv", index_col = 'Unnamed: 0')

In [ ]:
df_20_agents = pd.read_csv("../data/processed/results_20_agents.csv",index_col = 'Unnamed: 0')

In [ ]:
df_10_agents = pd.read_csv("../data/processed/results_10_agents.csv",index_col = 'Unnamed: 0')

In [ ]:
df_4_agents = pd.read_csv("../data/processed/results_4_agents.csv",index_col = 'Unnamed: 0')

In [ ]:
plt.rcParams['savefig.facecolor'] = (46/255., 48/255., 55/255.)
plt.rcParams['axes.facecolor'] = (46/255., 48/255., 55/255.)
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.spines['bottom'].set_color("white")
ax.spines['top'].set_color("white") 
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.title.set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
fig.set_facecolor((46/255., 48/255., 55/255.))
ax.set_facecolor((46/255., 48/255., 55/255.))


plt.scatter(df_4_agents['Size of the grid'],df_4_agents['Time'].values/60, label = "4 trains",alpha= 0.8)
plt.scatter(df_10_agents['Size of the grid'],df_10_agents['Time'].values/60, label = "10 trains",alpha = 0.8)
plt.scatter(df_20_agents['Size of the grid'],df_20_agents['Time'].values/60, label = "20 trains",alpha = 0.8)


plt.xlabel("size of the grid (number of cells)")

x1 = [20**2,40**2,50*50,60*60]
squad = ['(20x20)','(40x40)','(50,50)','(60x60)']

ax.set_xticks(x1)
ax.set_xticklabels(squad)

plt.ylabel("time until solution (min)")
#plt.title("Experiment of multicommodity flow formulation until memory error (model > 40 Gb)")

l = plt.legend()
for text in l.get_texts():
    text.set_color("white")



ax = plt.gca()
plt.tight_layout()
plt.savefig("../data/processed/time.png", dpi = 300)